In [7]:
import pandas as pd

df = pd.read_csv('base_calculation_file.csv')
df['Spend_lower'] = df['Spend_lower'].astype(str).str.replace(',', '').astype(float)
df['Spend_upper'] = df['Spend_upper'].astype(str).str.replace(',', '').astype(float)

#generating the elasticity dataset
channel_arr = [[]]
channels = df['Channel'].unique()
for channel in channels:
    # print (brand)
    s_lower = df[df['Channel']==channel]['Spend_lower'].iloc[0].astype(int)
    s_upper = df[df['Channel']==channel]['Spend_upper'].iloc[0].astype(int)
    m = df[df['Channel']==channel]['m'].iloc[0]
    c = df[df['Channel']==channel]['c'].iloc[0]
    
    for spend in range (s_lower,s_upper,30000):
        sales = spend*m + c
        channel_arr.append([channel,spend,sales])       
    df_pde = pd.DataFrame(channel_arr)
df_pde.rename(columns = {0:'Channel',1:'Spend',2:'Sales'},inplace=True)

#Calculate ROI
df_pde['ROI'] = round(df_pde['Sales']/df_pde['Spend'],2)
df_pde['Sales'] = round(df_pde['Sales'],2)
df_pde.dropna(inplace=True)
df_pde.sample(10)

,Channel,Spend,Sales,ROI
203,Paid Search,6338715.0,57447243.79,9.06
236,Paid Social,719036.0,6258370.68,8.70
325,Paid Social,3389036.0,29898550.68,8.82
160,Paid Search,5048715.0,45743073.79,9.06
413,Email,1039357.0,8877751.96,8.54
431,Email,1579357.0,13213411.96,8.37
269,Paid Social,1709036.0,15023830.68,8.79
406,Email,829357.0,7191661.96,8.67
10,Paid Search,548715.0,4914573.79,8.96
218,Paid Search,6788715.0,61530093.79,9.06


In [9]:
#we will pivot this table such that all 3 channel spends are combined with each other(650 X 500 X 320 )
from itertools import product


# Create a dictionary which will have the universe of values for each brand
channels = df_pde['Channel'].unique()
dfs = {channel: df_pde[df_pde['Channel'] == channel].reset_index(drop=True) for channel in channels}
dfs

{'Paid Search':          Channel      Spend        Sales   ROI
 0    Paid Search   278715.0   2464863.79  8.84
 1    Paid Search   308715.0   2737053.79  8.87
 2    Paid Search   338715.0   3009243.79  8.88
 3    Paid Search   368715.0   3281433.79  8.90
 4    Paid Search   398715.0   3553623.79  8.91
 ..           ...        ...          ...   ...
 213  Paid Search  6668715.0  60441333.79  9.06
 214  Paid Search  6698715.0  60713523.79  9.06
 215  Paid Search  6728715.0  60985713.79  9.06
 216  Paid Search  6758715.0  61257903.79  9.06
 217  Paid Search  6788715.0  61530093.79  9.06
 
 [218 rows x 4 columns],
 'Paid Social':          Channel      Spend        Sales   ROI
 0    Paid Social   209036.0   1742830.68  8.34
 1    Paid Social   239036.0   2008450.68  8.40
 2    Paid Social   269036.0   2274070.68  8.45
 3    Paid Social   299036.0   2539690.68  8.49
 4    Paid Social   329036.0   2805310.68  8.53
 ..           ...        ...          ...   ...
 159  Paid Social  4979036.0  4

In [17]:
# Get row indices for each brand df
channel_rows = [list(range(len(dfs[channel]))) for channel in channels]
len(channel_rows[0]), len(channel_rows[1]), len(channel_rows[2])

(218, 164, 109)

In [11]:
# Cartesian product of row indices
all_combinations = list(product(*channel_rows))
# len(all_combinations) ###653*490*327 --> 10,46,30,190
# type(all_combinations)--> list
all_combinations[0]

(0, 0, 0)

In [13]:
from tqdm.notebook import tqdm

progress_bar = tqdm(
    all_combinations,
    desc="Combining rows",
    bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}, {postfix}]"
)

# For each combination of row indices, build a row by horizontally joining brand rows
combined_rows = []
for row_idxs in all_combinations:

    row_parts = [dfs[channel].iloc[[idx]].reset_index(drop=True) for channel, idx in zip(channels, row_idxs)]
    #row_part_1 = dfs['Paid Search'].iloc[[0]] --> Paid Search	278715.0	2464863.79	8.84
    #row_pat_2 = dfs['Paid Social'].iloc[[0]] --> Paid Social	209036.0	1742830.68	8.34
    #row_pat_3 = dfs['Email'].iloc[[0]] -->	       Email	    139357.0	1651651.96	11.85

    combined_row = pd.concat(row_parts, axis=1)
    #combined_row =  Paid Search	278715.0	2464863.79	8.84 Paid Social	209036.0	1742830.68	8.34 Email	139357.0	1651651.96	11.85
    combined_rows.append(combined_row)

Combining rows:   0%|          | 0/847308 [00:00<?, ?it/s, ]

In [15]:
# Combine all rows into the final dataframe
final_df = pd.concat(combined_rows, axis=0).reset_index(drop=True)

# Optional: Clean up column names
new_cols = []
for channel in channels:
    new_cols.extend([f"{col}_{channel}" for col in dfs[channel].columns])
final_df.columns = new_cols

final_df

,Channel_Paid Search,Spend_Paid Search,Sales_Paid Search,ROI_Paid Search,Channel_Paid Social,Spend_Paid Social,Sales_Paid Social,ROI_Paid Social,Channel_Email,Spend_Email,Sales_Email,ROI_Email
0,Paid Search,278715.0,2464863.79,8.84,Paid Social,209036.0,1742830.68,8.34,Email,139357.0,1651651.96,11.85
1,Paid Search,278715.0,2464863.79,8.84,Paid Social,209036.0,1742830.68,8.34,Email,169357.0,1892521.96,11.17
2,Paid Search,278715.0,2464863.79,8.84,Paid Social,209036.0,1742830.68,8.34,Email,199357.0,2133391.96,10.70
3,Paid Search,278715.0,2464863.79,8.84,Paid Social,209036.0,1742830.68,8.34,Email,229357.0,2374261.96,10.35
4,Paid Search,278715.0,2464863.79,8.84,Paid Social,209036.0,1742830.68,8.34,Email,259357.0,2615131.96,10.08
...,...,...,...,...,...,...,...,...,...,...,...,...
847303,Paid Search,4178715.0,37849563.79,9.06,Paid Social,3119036.0,27507970.68,8.82,Email,1969357.0,16344721.96,8.30
847304,Paid Search,4178715.0,37849563.79,9.06,Paid Social,3119036.0,27507970.68,8.82,Email,1999357.0,16585591.96,8.30
847305,Paid Search,4178715.0,37849563.79,9.06,Paid Social,3119036.0,27507970.68,8.82,Email,2029357.0,16826461.96,8.29
847306,Paid Search,4178715.0,37849563.79,9.06,Paid Social,3119036.0,27507970.68,8.82,Email,2059357.0,17067331.96,8.29


In [81]:
# Sum Sales and Spend columns
def sum_columns_by_prefix(df, prefix, new_col_name):
    cols = [col for col in df.columns if col.startswith(prefix)]
    df[new_col_name] = df[cols].sum(axis=1)
    return df

# Apply transformations for Spend
final_df = sum_columns_by_prefix(final_df, "Spend_", "Total_Spend")
final_df = sum_columns_by_prefix(final_df, "Sales_", "Total_Sales")


# Average ROI
def mean_columns_by_prefix(df, prefix, new_col_name):
    cols = [col for col in df.columns if col.startswith(prefix)]
    df[new_col_name] = round(df[cols].apply(pd.to_numeric, errors='coerce').mean(axis=1),2)
    return df

final_df = mean_columns_by_prefix(final_df, "ROI_", "Avg_ROI")

final_df['Net_ROI'] = round(final_df['Total_Sales']/final_df['Total_Spend'],2)

fina_df_sample = final_df.sample(100000)
fina_df_sample.to_csv('mmm_universe_of_combination_sample.csv',index=False)